In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

from pydantic import BaseModel, Field
from typing import List, Dict, TypedDict, Set, Union

from groq import Groq

In [17]:
def llm_call(messages: List, tools: Union[List, None], response_schema: Union[BaseModel, Dict], model: str = "openai/gpt-oss-20b"):
    # normalize response_schema to a JSON schema dict expected by Groq
    if isinstance(response_schema, dict):
        schema = response_schema
    elif isinstance(response_schema, type) and issubclass(response_schema, BaseModel):
        schema = response_schema.model_json_schema()
    else:
        raise TypeError("response_schema must be either a pydantic BaseModel subclass or a JSON schema dict")
    return Groq().chat.completions.create(
        messages=messages, model=model, tools=tools,
        temperature=0.8,    # Need more creativity
        response_format={
            "type": "json_object",
            # "json_schema": schema
        }, # type: ignore
        stream=False,
    ) # type: ignore

In [18]:
class SimpleSchema(BaseModel):
    code: str = Field(..., description="The Manim generation code")
    

response = llm_call([
    {"role": "user", "content": "Generate a simple Manim code to visualize a fully connected Neural Network. Return JSON format data like this: {'code': 'Manim code'}"}
    ],
    tools=None,
    response_schema={"code": "Manim code"}
)

In [19]:
import json
code = json.loads(response.choices[0].message.content)['code']
print(code)

from manim import *

class FullyConnectedNN(Scene):
    def construct(self):
        # Define layers
        layers = [3, 4, 2]  # number of neurons per layer
        neurons = []
        for i, count in enumerate(layers):
            layer = VGroup(*[Circle(radius=0.2).shift(LEFT*4 + RIGHT*i + UP*(count-1)/2 + DOWN*j) for j in range(count)])
            neurons.append(layer)
        for layer in neurons:
            self.add(layer)
        # Connect layers
        for i in range(len(neurons)-1):
            for n1 in neurons[i]:
                for n2 in neurons[i+1]:
                    line = Line(n1.get_center(), n2.get_center(), buff=0.05).set_color(BLUE)
                    self.add(line)
        self.wait()


In [ ]:
import subprocess
import tempfile
import os

cwd = os.path.abspath(os.path.curdir) + "/tmp/"
os.makedirs(cwd, exist_ok=True)

with tempfile.TemporaryFile(mode="w+", suffix=".py", dir=cwd, delete=False) as file:
    file.write(code)
    
res = subprocess.run(["manim", "-p", file.name], cwd=cwd, capture_output=True)
os.remove(file.name)

In [21]:
print((res.stderr).decode())

In [ ]:
import glob
from IPython.display import Video, display
import shutil

save_dir = os.path.abspath(os.path.curdir) + "/videos/"
os.makedirs(save_dir, exist_ok=True)

mp4_files = glob.glob("**/*.mp4", root_dir=cwd, recursive=True)
if mp4_files:
    display(Video(filename=os.path.join(cwd, mp4_files[0]), embed=True))
    shutil.move(os.path.join(cwd, mp4_files[0]), os.path.join(save_dir, mp4_files[0].split("\\")[-1]))
else:
    print("No .mp4 files found.")

In [33]:
res = subprocess.run(["rm -rf", os.path.abspath(os.path.curdir) + "/tmp/"], shell=True)

In [34]:
res

CompletedProcess(args=['rm -rf', 'c:\\Users\\Utkarsh\\OneDrive\\Documents\\GitHub\\visual-explainer\\tests/tmp/'], returncode=1)